In [4]:
import pandas as pd
import numpy as np

In [118]:
from cluster_functions import load_vdjdb
import os
vdjdb = load_vdjdb(os.path.join(os.getcwd(),'data/vdjdb_full.txt'))

In [28]:
nodelist = '/Users/danhudson/Documents/Academic/Oxford/Oxford_DPhil/Software/clusTCR/tcr_compare/cytoscape/20220729_101706_typhi.csv/clustcr_beta_typhi.csv_nodelist.txt'

In [31]:
nodes = pd.read_csv(nodelist)

In [32]:
nodes

,CDR3,cluster,basicity,hydrophobicity,helicity,mutation stability,motif,V,d.beta,J,...,species,mhc.a,mhc.b,mhc.class,Epitope,antigen.gene,antigen.species,condition,subject,Length
0,CASSLEGGTYEQYF,0,209.871429,-0.195714,1.060000,22.285714,CASSL.aG.yEQYF,TRBV7-9*01,NaN,TRBJ2-7*01,...,HomoSapiens,HLA-A*02,B2M,MHCI,HLA-A*02_NLVPMVATV_pp65_CMV,pp65,CMV,NaN,NaN,14
1,CASSLSGGTYEQYF,0,209.300000,-0.292143,1.070714,22.142857,CASSL.aG.yEQYF,TRBV11-1,TRBD2,TRBJ2-7,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,14
2,CASSLLAGAYEQYF,0,209.328571,0.587857,1.107857,23.000000,CASSL.aG.yEQYF,TRBV5-6,TRBD2,TRBJ2-7,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,14
3,CASSLVAGAYEQYF,0,209.264286,0.463571,1.107143,21.785714,CASSL.aG.yEQYF,TRBV7-9,TRBD1,TRBJ2-7,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,14
4,CASSLLAGAGEQYF,0,208.585714,0.208571,1.110714,22.714286,CASSL.aG.yEQYF,TRBV12-4,TRBD2,TRBJ2-7,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61306,CASSLWGQGLNYGYTF,7960,209.262500,0.175000,1.098750,25.625000,CASSlWGQGsNYGYTF,TRBV27,TRBD1,TRBJ1-2,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,16
61307,CASSLGGGKANYGYTF,7966,208.700000,-0.764375,1.096250,23.437500,CASSLGQG[LK]ANYGYTF,TRBV27,TRBD1,TRBJ1-2,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,16
61308,CASSLGQGKANYGYTF,7966,209.418750,-0.730000,1.084375,22.375000,CASSLGQG[LK]ANYGYTF,TRBV13,TRBD1,TRBJ1-2,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,16
61309,CASSLGQGLANYGYTF,7966,208.656250,-0.120000,1.109375,23.000000,CASSLGQG[LK]ANYGYTF,TRBV27*01,NaN,TRBJ1-2*01,...,HomoSapiens,HLA-B*08,B2M,MHCI,HLA-B*08_FLKEKGGL_Nef_HIV-1,Nef,HIV-1,NaN,AC-14,16


In [43]:
from collections import Counter

clusters = nodes['cluster'].unique()
counts = {k:v for k,v in Counter(nodes['Epitope'].values).items()}
freqs_global = {ep:counts[ep]/len(nodes) for ep in counts.keys()}
outdict={}
for cluster in clusters:
    sub = nodes[nodes['cluster']==cluster]
    eps = {k:v for k,v in Counter(sub['Epitope'].values).items()}
    freqs_local = {ep:eps[ep]/len(sub) for ep in eps.keys()}
    outdict[cluster]={}
    outdict[cluster]['Counts']=eps
    outdict[cluster]['Enrichment']={epitope: freqs_local[epitope]/freqs_global[epitope] for epitope in freqs_local.keys()}

In [45]:
for cluster in nod
tdict=outdict[0]['Enrichment']
cmax = max(tdict, key=tdict.get)

In [46]:
cmax

'HLA-B*07:02_RPPIFIRRL_EBNA3A_EBV'

In [22]:
sub = [nodes[nodes['cluster']==c] for c in nodes['cluster'].unique() if 'User Input' in nodes[nodes['cluster']==c]['Source'].unique()]

In [23]:
sub2 = pd.DataFrame(pd.concat(sub))

In [24]:
sub2

,CDR3,cluster,basicity,hydrophobicity,helicity,mutation stability,motif,V,d.beta,J,...,species,mhc.a,mhc.b,mhc.class,Epitope,antigen.gene,antigen.species,condition,subject,Length
0,CASSLSFGDTEAFF,0,209.021429,0.403571,1.093571,21.357143,CASSL..G.TEAFF,TRBV12-4*01,NaN,TRBJ1-1*01,...,HomoSapiens,HLA-A*03:01,B2M,MHCI,HLA-A*03:01_KLGGALQAK_IE1_CMV,IE1,CMV,NaN,1,14
1,CASSLSTGDTEAFF,0,208.992857,-0.030714,1.081429,19.857143,CASSL..G.TEAFF,TRBV5-1,TRBD1,TRBJ1-1,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,14
2,CASSLADGGTEAFF,0,208.264286,0.025000,1.102857,21.214286,CASSL..G.TEAFF,TRBV7-9,.,TRBJ1-1,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,14
3,CASSLARGGTEAFF,0,210.292857,0.005000,1.107143,21.642857,CASSL..G.TEAFF,TRBV5-1*01,NaN,TRBJ1-1*01,...,HomoSapiens,HLA-A*02,B2M,MHCI,HLA-A*02_GLCTLVAML_BMLF1_EBV,BMLF1,EBV,NaN,Donor0439,14
4,CASSLGAGRTEAFF,0,210.292857,0.005000,1.107143,21.642857,CASSL..G.TEAFF,TRBV11-2*01,NaN,TRBJ1-1*01,...,HomoSapiens,HLA-A*02,B2M,MHCI,HLA-A*02_SLYNTVATL_Gag_HIV-1,Gag,HIV-1,NaN,6,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71359,CASSLVGRGLGYGYTF,7974,209.593750,0.088125,1.123750,25.000000,CASSLVG[RE]GLGYGYTF,TRBV11-2,TRBD1,TRBJ1-2,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,16
71360,CASSQADRGANYGYTF,7975,210.537500,-0.826875,1.070000,19.625000,CASS[RQ]ADRGANYGYTF,TRBV14,TRBD1,TRBJ1-2,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,16
71361,CASSRADRGANYGYTF,7975,211.962500,-0.827500,1.069375,20.062500,CASS[RQ]ADRGANYGYTF,TRBV4-3,TRBD1,TRBJ1-2,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,16
71362,CASRLGQGATNYGYTF,7976,210.625000,-0.480000,1.094375,22.000000,CASRLGQGA[TY]NYGYTF,TRBV19,TRBD1,TRBJ1-2,...,NaN,NaN,NaN,NaN,Orphan TCR,NaN,NaN,NaN,NaN,16


In [13]:
nodes['cluster'].unique()

array([   0,    1,    2, ..., 7974, 7975, 7976])

In [10]:
nodes['Epitope'].unique()

array(['HLA-A*11:01_GTSGSPIVNR_NS3_DENV1', 'Orphan TCR',
       'HLA-A*02_ELAGIGILTV_MLANA_HomoSapien',
       'HLA-A*03:01_KLGGALQAK_IE1_CMV',
       'HLA-DRA*01_GLIYNRMGAVTTEV_M1_InfluenzaA',
       'HLA-A*02_NLVPMVATV_pp65_CMV', 'HLA-A*02_GILGFVFTL_M_InfluenzaA',
       'HLA-B*27:05_KRWIIMGLNK_Gag_HIV-1',
       'HLA-A*11:01_AVFDRKSDAK_EBNA4_EBV',
       'HLA-A*02:01_ELAGIGILTV_MLANA_HomoSapien',
       'HLA-A*02:01_LLWNGPMAV_NS4B_YFV',
       'HLA-B*27:05_KRWIILGLNK_Gag_HIV-1',
       'HLA-DRA1*01_LLQTGIHVRVSQPSL_pp65_CMV',
       'HLA-A*01:01_VTEHDTLLY_pp50_CMV',
       'HLA-A*02:01_GILGFVFTL_M_InfluenzaA',
       'HLA-A*11:01_GTSGSPIINR_NS3_DENV3/4',
       'HLA-B*08:01_RAKFKQLL_BZLF1_EBV',
       'HLA-A*02:01_FLYNLLTRV_SEC24A_HomoSapien',
       'HLA-DRA*01_QARQMVQAMRTIGTHP_M1_InfluenzaA',
       'HLA-DRA*01_PKYVKQNTLKLAT_HA_InfluenzaA',
       'HLA-A*02:01_NLVPMVATV_pp65_CMV',
       'HLA-B*42:01_FPRPWLHGL_Vpr_HIV-1', 'HLA-A*02_GLCTLVAML_BMLF1_EBV',
       'HLA-A*03:01_RLRAEAQV